In [ ]:
import mqr
from mqr.plot import Figure
from mqr.nbtools import hstack, vstack

# Datasets

In [ ]:
import pandas as pd

# Random glue data
data = pd.read_csv(mqr.sample_data('anova-glue.csv'), index_col='Run')

---
# ANOVA

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [ ]:
# Simple one factor
model = ols('adhesion_force ~ C(primer) + C(glue) + C(primer):C(glue)', data=data)
result = model.fit()

vstack(
    mqr.anova.adequacy(result),
    mqr.anova.summary(result),
)

In [ ]:
display(mqr.anova.groups(data, value='adhesion_force', factor='primer'))
display(mqr.anova.groups(data, value='adhesion_force', factor='glue'))

---
## Residual analysis

In [ ]:
# See https://en.wikipedia.org/wiki/Studentized_residual#Internal_and_external_studentization

influence = result.get_influence()
N = influence.nobs
with Figure(8, 4) as (fig, ax):
    ax.plot(influence.resid_studentized_internal, linewidth=0, marker='o', fillstyle='none')
    ax.plot(influence.resid_studentized_external, linewidth=0, marker='x')
    ax.set_ylabel('residuals')

    axt = ax.twinx()
    axt.bar(range(N), 1-influence.cooks_distance[1], alpha=0.5)
    axt.set_ylim(0.0, 1.0)
    axt.set_ylabel("Cook's distance (1-p)")

    ax.legend(['stud. internal', 'stud. external'])
    ax.grid(axis='y')

In [ ]:
# NB: Factor plots are added manually. See how a 2*2 slice of the axes `ax[:2, :2]` is passed to the `residuals` function.
#     Any 2*2 slice will work. The remaining factor plots are added manually into ax[2, 0] and ax[2, 1].
with Figure(8, 6, 3, 2, height_ratios=[3, 2, 2]) as (fig, ax):
    mqr.plot.regression.residuals(result, tr='studentised_external', influence_stat='cooks_dist', axs=ax[:2, :2])
    mqr.plot.regression.res_v_factor(data['primer'], result, ax[2, 0], factor_name='primer')
    mqr.plot.regression.res_v_factor(data['glue'], result, ax[2, 1], factor_name='glue')